# 常规赛：遥感影像地块分割

本程序使用PaddleSeg进行分割，特别感谢[https://aistudio.baidu.com/aistudio/projectdetail/2276682]，提供了调用PaddleSeg的示例。

## 准备数据

In [ ]:
! unzip -oq /home/aistudio/data/data77571/train_and_label.zip -d work/

## 导入包

In [1]:
!pip install paddleseg

#导入常用的库
import os
import random
import numpy as np
from random import shuffle
import cv2
import paddle
from PIL import Image
import shutil
import re
from paddle.vision.transforms import functional as F
import os.path
import paddleseg.transforms as T
from paddleseg.datasets import OpticDiscSeg,Dataset
import paddleseg.models
from paddleseg.models import UNet
from paddleseg.models import OCRNet
from paddleseg.models.losses import CrossEntropyLoss,DiceLoss, MixedLoss
from paddleseg.core import train
from paddleseg.core import evaluate
from paddleseg.core import predict
from PIL import Image

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


## 数据分割

In [2]:
names=[item.split('.')[0] for item in os.listdir('work/img_train')]
ori_train_list=[]
img_dir='/home/aistudio/work/img_train/'
mask_dir='/home/aistudio/work/lab_train/'
for name in names:
    if os.path.exists(img_dir+name+'.jpg') and os.path.exists(mask_dir+name+'.png'):
        ori_train_list.append(img_dir+name+'.jpg '+mask_dir+name+'.png'+'\n')

In [ ]:
cut_percent=0.2
cut_point=int(cut_percent*len(ori_train_list))
random.shuffle(ori_train_list)

with open('train.txt','w') as f:
    for item in ori_train_list[cut_point:]:
        f.write(item)

with open('eval.txt','w') as f:
    for item in ori_train_list[:cut_point]:
        f.write(item)

## 进行训练

由于本数据都较为规整，因此几乎不需要resize

In [3]:
train_transforms = [
    #T.RandomHorizontalFlip(),# 水平翻转
    #T.RandomVerticalFlip(),# 垂直翻转
    #T.RandomDistort(0.4),# 随机扭曲
    #T.RandomBlur(0.3),# 高斯模糊
    #T.RandomScaleAspect(min_scale=0.5,aspect_ratio=0.5),# 随机缩放
    T.Normalize()  # 图像标准化
]

val_transforms = [
    T.Normalize()
]

In [4]:
# 构建训练集
train_dataset = Dataset(
    dataset_root='/home/aistudio',
    train_path='/home/aistudio/train.txt',
    transforms=train_transforms,
    num_classes=4,
    mode='train'
)
#验证集
val_dataset = Dataset(
    dataset_root='/home/aistudio',
    val_path='/home/aistudio/eval.txt',
    transforms=val_transforms,
    num_classes=4,
    mode='val'
)

In [5]:
model = UNet(num_classes=4)#使用unet进行训练

In [9]:
base_lr =0.001
iters = 10000

#自动调整学习率
lr =paddle.optimizer.lr.CosineAnnealingDecay(base_lr, T_max=(iters // 3), last_epoch=0.5)
u_optimizer = paddle.optimizer.Adam(lr, parameters=model.parameters())

mixtureLosses = [CrossEntropyLoss(),DiceLoss() ]
mixtureCoef = [0.5,0.5]
losses = {}
losses['types'] = [MixedLoss(mixtureLosses, mixtureCoef)]
losses['coef'] = [1]

In [10]:
#进行训练
train(
    model = model,
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    optimizer=u_optimizer,
    save_dir='output/UNet',
    iters=iters,  
    batch_size=32, 
    save_interval=2000,
    log_iters=20,
    num_workers=2,
    losses=losses,
    # losses=CrossEntropyLoss(),
    use_vdl=False
    )

## 打包生成结果

In [ ]:
! unzip -oq /home/aistudio/data/data77571/img_test.zip -d work/

In [ ]:
test_list=["work/img_testA/"+item for item in os.listdir("work/img_testA")]

transforms = T.Compose([
    T.Normalize()
])

model = paddleseg.models.UNet(num_classes=4)

predict(
    model,
    #换自己保存的模型文件
    model_path = 'output/UNet/best_model/model.pdparams',
    transforms=transforms,
    image_list=test_list,
    save_dir='result',)

2021-09-25 19:41:58 [INFO]	Loading pretrained model from output/UNet/best_model/model.pdparams
2021-09-25 19:41:58 [INFO]	There are 112/112 variables loaded into UNet.
2021-09-25 19:41:58 [INFO]	Start to predict...


1375/4608 [=======>......................] - ETA: 1:

In [ ]:
! zip -q -r result.zip result/pseudo_color_prediction

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 